In [ ]:
import requests
import zipfile
from zarr.storage import LocalStore, ZipStore

from io import BytesIO

## Data download & extraction

In [2]:
zipped_zarr_url = "https://zenodo.org/records/13305156/files/20200812-CardiomyocyteDifferentiation14-Cycle1.zarr.zip"

In [ ]:
r = requests.get(zipped_zarr_url)
r.raise_for_status()
buf = BytesIO(r.content)
with zipfile.ZipFile(buf) as z:
    z.extractall("../data")

## DirectoryStore -> ZipStore conversion

In [ ]:
source_zarr_file = "../data/20200812-CardiomyocyteDifferentiation14-Cycle1.zarr"
dest_zarr_file = "../data/20200812-CardiomyocyteDifferentiation14-Cycle1.zarr.zip"

In [ ]:
local_store = LocalStore(source_zarr_file, read_only=True)

In [ ]:
# # Doesn't work: need to use ZipStore.open instead
# # AttributeError: 'ZipStore' object has no attribute '_lock'
# with ZipStore(dest_zarr_file, mode="w", compression=zipfile.ZIP_STORED) as zip_store:  
#     pass

In [ ]:
# # Doesn't work: no context manager support for ZipStore?
# # TypeError: 'coroutine' object does not support the context manager protocol
# with ZipStore.open(dest_zarr_file, mode="w", compression=zipfile.ZIP_STORED) as zip_store:
#     pass

In [ ]:
zip_store = ZipStore(dest_zarr_file, mode="w", compression=zipfile.ZIP_STORED)
zip_store.open() # don't forget to close!

In [ ]:
# # Doesn't work: convenience "copy routines" are not implemented
# # NotImplementedError
# zarr.copy_store(local_store, zip_store)

In [ ]:
async def copy_keys(source_store, dest_store):
    async for key in source_store.list():
        value = await source_store.get(key)
        await dest_store.set(key, value)

await copy_keys(local_store, zip_store)

In [ ]:
zip_store.close()